In [1]:
from skopt import Optimizer
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import RBF, ConstantKernel, Product
from tqdm import tqdm_notebook as tqdm
from skopt import gp_minimize
from time import sleep
import docker

### Глобальные Параметры

In [2]:
# первые n_initial_points модель не обучается
n_initial_points = 5

# число итераций цикла
n_calls = 3

# оптимизация на кубе [low_constraint, high_constraint]^dim
low_constraint, high_constraint = 10., 420.
dim = 2

# столько контейнеров вызываются для параллельной работы
batch_size = 2

# директория на сервере, хранит директории, которые будут монтироваться в контейнеры
folder_local = '/home/matyushinleonid/lhcb_ecal/feb_meeting/folder_local'

# директория для файлов input и output внутри контейнера
folder_container = '/home/nb_user/logs'

# python-клиент докера
client = docker.from_env()

# имя образа
container = "calorbuild"

# имена директорий, каждая соответствует своей копии образа
worker_names = ['first_worker', 'second_worker']

### Модель

In [3]:
kernel = Product(ConstantKernel(1), RBF(1)) + ConstantKernel(1)

model = GaussianProcessRegressor(alpha=0, 
                                 normalize_y=True, 
                                 noise='gaussian', 
                                 n_restarts_optimizer=10, 
                                 kernel=kernel)

optimizer = Optimizer([[low_constraint, high_constraint]]*dim,
                      model,
                      n_initial_points=n_initial_points,
                      acq_func='EI',
                      acq_optimizer='lbfgs',
                      random_state=None)

### Оптимизация (+ работа с контейнерами)

In [4]:
def write_input_file(worker_name, input_data):
    file_to_write = '{}/{}/input.txt'.format(folder_local, worker_name)
    string_to_write = ' '.join(map(str, input_data))
    with open(file_to_write, "w") as file:
        print(string_to_write,
              file=file)

def create_job(worker_name):
    folder_to_mount = '{}/{}'.format(folder_local, worker_name)
    client.containers.run(container,
                          privileged=True,
                          remove=True,
                          detach=True,
                          hostname='dev',
                          tty=True,
                          stdin_open=True,
                          volumes={folder_to_mount: {'bind': folder_container,
                                                     'mode': 'rw'}})

def read_output_file(worker_name):
    file_to_read = '{}/{}/output.txt'.format(folder_local, worker_name)
    with open(file_to_read, 'r') as myfile:
        data = myfile.read()
    return float(data)

In [5]:
for i in tqdm(range(n_calls)):
    X = optimizer.ask(n_points=batch_size)
    for i, worker_name in enumerate(worker_names):
        x = X[i]
        write_input_file(worker_name, x)
        create_job(worker_name)
    
    sleep(3 * 60)
    
    Y = []
    for i, worker_name in enumerate(worker_names):
        y = read_output_file(worker_name)
        Y.append(y)
        
    optimizer.tell(X, Y)

Widget Javascript not detected.  It may not be installed or enabled properly.


# Parallel with multiproccesing



In [34]:
from multiprocessing import Queue, Pool, Manager

In [ ]:
def worker_do_job(q_in, q_out):
    while True:
        data = q_in.get()
        create_job() #??        
        result = read_output_file() #?? Do one file
        q_out.put(result)        
    return     

In [ ]:
pool = Pool(batch_size) 
m = Manager()
q_in = m.Queue()
q_out = m.Queue()
pool.starmap_async(worker_do_job, [(q_in, q_out)]*batch_size)

X = optimizer.ask(n_points=batch_size)
for i in range(batch_size):
    inp = write_input_file(worker_name, X[i]) #??
    q_in.put(inp)

for i in tqdm(range(n_calls-batch_size)): 
    x, y  = q_out.get()
    optimizer.tell(x, y)
    
    inp_path_file = write_input_file(optimizer.ask()) #??
    q_in.put(inp_path_file)
    
for i in range(batch_size):
    x, y  = q_out.get()
    optimizer.tell(x, y)        
    
pool.terminate()